# Formatted zone

In [ ]:
# ! pip install duckdb --user --upgrade --quiet
# ! pip install pyarrow --user --upgrade --quiet

In [ ]:
import duckdb 
import pyarrow as pa
import numpy as np

In [ ]:
def columns(name_DB):
    con =  duckdb.connect(f'{name_DB}.duckdb')
    c1 = con.execute(f"SELECT Column_Name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = N'{name_DB}18'").arrow()
    cols1 = c1.column(0).to_pylist()

    c2 = con.execute(f"SELECT Column_Name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = N'{name_DB}19'").arrow()
    cols2 = c2.column(0).to_pylist()

    c3 = con.execute(f"SELECT Column_Name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = N'{name_DB}20'").arrow()
    cols3 = c3.column(0).to_pylist()
    return cols1, cols2, cols3

def join_table(cols1,cols2,cols3):
    v18 = [f'v18.{x} AS {x}_2018' for x in cols1]
    v19 = [f'v19.{x} AS {x}_2019' for x in cols2]
    v20 = [f'v20.{x} AS {x}_2020' for x in cols3]
    all = np.concatenate([v18,v19,v20])
    columns = ', '.join(all)
    return columns

def join_household():
    cols1, cols2, cols3 = columns('household')
    str_join = join_table(cols1[1:],cols2[1:],cols3[1:]) # skip section

    con = duckdb.connect(f'household.duckdb')
    query = f"CREATE TABLE household AS SELECT v18.section, {str_join} FROM household18 v18 "
    query += "FULL OUTER JOIN household19 v19 ON v18.section = v19.section "
    query += "FULL OUTER JOIN household20 v20 ON v19.section = v20.section"
    df = con.execute(query)
    con.close()

def join_nationalities():
    cols1, cols2, cols3 = columns('nationalities')
    str_join = join_table(cols1[2:],cols2[2:],cols3[2:]) # skip code and madrid

    con = duckdb.connect(f'nationalities.duckdb')
    query = f"CREATE TABLE nationalities AS SELECT v18.Code, v18.Madrid, {str_join} FROM nationalities18 v18 "
    query += "FULL OUTER JOIN nationalities19 v19 ON v18.Code = v19.Code AND v18.Madrid = v19.Madrid "
    query += "FULL OUTER JOIN nationalities20 v20 ON v19.Code = v20.Code AND v18.Madrid = v19.Madrid"
    df = con.execute(query)
    con.close()

def formatted_zone():
    join_household()
    join_nationalities()
    

In [ ]:
formatted_zone()